# Weekly exercise 10: Markov employment transitions

Consider a worker who, at any given time can be either

- employed full time (x = 2)  
- employed part time (x = 1)  
- unemployed         (x = 0)  


Assume that an unemployed worker finds a part time job with probability $ \alpha $ and full time job with probability $ \beta < \alpha $.

A full time worker may loose their job, or transfer to part time job, with probability $ \gamma $.

Further, a part time worker may loose their job, or finds a full time job, with probability $ \delta $.

## Tasks:

1. Write a function that would return the transition probability matrix given the parameters $ (\alpha,\beta,\gamma,\delta) $ after making the appropriate checks  
1. Assume that initially half of the population has no job, while another half has a full time job.  Simulate the distribution of employment states in the following 50 time periods.  Use what you think are appropriate values of the parameters.  
1. Visualize the evolution of the employment states using an area plot  
1. Compute the stationary distribution of employment states.  Verify that it coincides with the simulated invariant distribution.  
1. Consider an initially unemployed worker.  Simulate employment states for this worker for $ N $ periods.  Compute the fractions of time the worker is in each state, and make a line plot of the three of these fraction with N on the horizontal axis.  Verify that the ergodicity of the invariant distribution.  


In your work, do not use other than standard scientific libraries (like NumPy and Pandas).
The exercise is inspired by QuantEcon example on [https://python.quantecon.org/finite_markov.html](https://python.quantecon.org/finite_markov.html) which you may find helpful to go though.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]

def TransProbMtrx(α=0.,β=0.,γ=0.,δ=0.):
    '''Returns transision probability matrix after the check for parameters in range
    '''
    M = np.asarray([[1-α-β, α,     β     ],
                    [δ,     1-2*δ, δ     ],
                    [γ,     γ,     1-2*γ ]])
    assert all(0<=M[0]) and all(M[0]<=1), 'Parameters α and β lead to bad transition probabilities'
    assert all(0<=M[1]) and all(M[1]<=1), 'Parameter δ leads to bad transition probabilities'
    assert all(0<=M[2]) and all(M[2]<=1), 'Parameter γ leads to bad transition probabilities'
    return M

α,β,γ,δ = 0.075,0.05,0.01,0.02
M = TransProbMtrx(α,β,γ,δ)
print(M)

In [ ]:
# The task is not making completely clear the difference between simulating and computing theoretical distributions.
# In this code the distinction is made clear.

def DrawDiscreteVector(probs,size=1):
    '''Draws n realizations of discrete random variables
    generated from given probability distibution (base 0)
    '''
    probs = np.asarray(probs)
    assert probs.ndim == 1, 'Expecting a one-dimensional array of probabilities'
    assert np.abs(probs.sum()-1)<1e-10, 'Passed probabilities {} do not sum up to one'.format(probs)
    cdf = np.cumsum(probs)  # cumulative distribution
    u = np.random.uniform(size=size)  # u(0,1)
    d = np.empty(shape=size,dtype=int)
    for i in reversed(range(cdf.size)):
        d[u<=cdf[i]] = i  # fill index, even if multiple times
    return d  # between i-1 and i values of cdf

ψ = np.array([.5,0,.5])
print(DrawDiscreteVector(ψ,15))  # simulate 15 realization from the same distribution
print(DrawDiscreteVector(ψ,(8,4)))  # actually we can do it for any shape

In [ ]:
def SimulateMarkovPopulation(P,psi,n=2,T=10,verbose=True):
    '''Simulates Markov chain with given transition probability matrix (P),
    initial state distribution (psi), a number of paths to simulate (n), and
    a number of steps (time periods) to simulate (T)
    '''
    P = np.asarray(P)  # convert to numpy array
    psi = np.asarray(psi)
    assert np.all(np.abs(P.sum(axis=1)-1)<1e-10), 'Passed P is not stochastic matrix'
    m = psi.size  # number of states in the chain
    # simulate the initial state
    X = np.empty((n,T+1),dtype=int)  # allocate space for the simulated values
    X[:,0] = DrawDiscreteVector(psi,size=n)  # initial values in first column
    if verbose:
        print('Simulating Markov chain with n={} observations for T={} periods'.format(n,T))
        print('t={:<3d} {} initial'.format(0,X[:,0])) if verbose else None
    for t in range(1,T+1): # main loop over time
        for i in range(m): # loop over states
            mask = X[:,t-1]==i  # rows equal to i at t-1
            X[mask,t] = DrawDiscreteVector(P[i,:],size=mask.sum())  # simulate using appropriate row of P
        print('t={:<3d} {}'.format(t,X[:,t])) if verbose else None
    # compute fractions of states
    F = np.empty((m,T+1),dtype=float)
    for i in range(m): # loop over states
        F[i,:] = np.sum(X==i,axis=0) / n
    return X,F

T=30
sim,fr = SimulateMarkovPopulation(M,ψ,n=10,T=T,verbose=True)
sim,fr = SimulateMarkovPopulation(M,ψ,n=100,T=T,verbose=False)

# make an area plot
plt.stackplot(range(T+1),fr,labels=['Unemployed','Employed Part-Time','Employed Full-time'])
plt.xlabel('Time', fontsize=12)
plt.ylabel('Distribution of the population in fraction of total population',fontsize=12)
plt.legend()

In [ ]:
def StationaryDistribution(P,psi0=[None,]):
    '''Computes stationary distribution for the Markov chain given by transition probability
    matrix P. Method: linear solver.
    '''
    if psi0[0]==None:
        # degenrate initial distribution
        psi0 = [0,]*P.shape[0]
        psi0[0]=1.0
    P,psi0 = np.asarray(P),np.asarray(psi0)  # convert to np arrays (in case lists were passed)
    assert np.all(np.abs(P.sum(axis=1)-1)<1e-10), 'Passed P is not stochastic matrix'
    assert np.abs(psi0.sum()-1)<1e-10, 'Passed probabilities do not sum up to one'
    m = P.shape[0]  # dimension of the problem
    A = np.ones((m+1,m+1))  # square matrix of ones
    A[:-1,:-1] = np.eye(m)-P.T
    b = np.ones(m+1)
    b[-1]=2
    res = np.linalg.solve(A,b)
    return res[:-1]

st = StationaryDistribution(M)
print('Stationary distribution is {}'.format(st))
print('Simulated distributions at T={} is {}'.format(T,fr[:,-1]))
print('The difference is {}'.format(np.abs(st-fr[:,-1])))
print('Conclusion: after simulating the distribution in a population of decent size for a number of the periods, the cross-sectional frequencies are close to the stationary distribution')

In [ ]:
N = 1000
sim,_ = SimulateMarkovPopulation(M,[1,0,0],n=1,T=N,verbose=False)  # all simulations
burnin = N//10  # skipping first periods for burn in
t = np.arange(burnin,N)
tfr = np.zeros((3,t.size))
for i,n in enumerate(t):
    tfr[:,i] = np.bincount(sim.ravel()[:n],minlength=3)/n  # counts of each value, after flattening into 1-dim array

# plot
fig, ax = plt.subplots(figsize=(12,8))
ax.plot(t,tfr.T)
ax.set_xlabel('Number of simulated periods', fontsize=12)
ax.set_ylabel('Fraction of times being in each state',fontsize=12)
ax.legend(('Unemployed','Employed Part-Time','Employed Full-time'))